In [1]:
import libtspenvv2o3 as libtsp
from pathlib import Path
from pprint import pprint
import numpy as np

In [2]:
blob = list(Path('.').rglob('tsp*15*.dat'))[0].read_text()
lines = list(blob.strip().split('\n'))

n_objs = int(lines[0].strip())
n_vars = int(lines[1].strip())
objs = []
_objs = []
count = 0
for line in lines[2:]:
    _objs.append(list(map(int, line.strip().split(" "))))
    count += 1
    
    if count == n_vars:
        objs.append(_objs)
        _objs = []
        count = 0

In [3]:
env = libtsp.TSPEnv()
env.reset(3)
env.set_inst(n_vars, n_objs, objs)
env.initialize_dd_constructor()
env.generate_dd()
env.compute_pareto_frontier()

Layer 1
Layer 2
Layer 3
Layer 4
Layer 5
Layer 6
Layer 7
Layer 8
Layer 9
Layer 10
Layer 11
Layer 12
Layer 13
Layer 14
Layer topdown: 0 - layer bottomup: 16
Layer topdown: 1 - layer bottomup: 16
Layer topdown: 1 - layer bottomup: 15
Layer topdown: 2 - layer bottomup: 15
Layer topdown: 3 - layer bottomup: 15
Layer topdown: 3 - layer bottomup: 14
Layer topdown: 4 - layer bottomup: 14
Layer topdown: 4 - layer bottomup: 13
Layer topdown: 5 - layer bottomup: 13
Layer topdown: 5 - layer bottomup: 12
Layer topdown: 6 - layer bottomup: 12
Layer topdown: 6 - layer bottomup: 11
Layer topdown: 7 - layer bottomup: 11
Layer topdown: 7 - layer bottomup: 10
Layer topdown: 8 - layer bottomup: 10
Layer topdown: 8 - layer bottomup: 9
Coupling layer : 9


0

In [4]:
frontier = env.get_frontier()
dd = env.get_dd()

In [5]:
for i, l in enumerate(dd):
    print(i, len(l))

0 1
1 14
2 182
3 1092
4 4004
5 10010
6 18018
7 24024
8 24024
9 18018
10 10010
11 4004
12 1092
13 182
14 14


In [6]:
X = frontier['x']
X = np.array(X)


In [7]:
X = X[:, 1:]

In [19]:
n_pareto_sol = X.shape[0]

In [21]:
n_pareto = 0
n_nodes = 0
for i in range(1, X.shape[1]+1):
    X_ = X[:, :i]
    states = np.zeros((X.shape[0], 15))
    ind = np.arange(states.shape[0])
    for j in range(X_.shape[1]):
        states[ind, X_[:, j]] = 1
    last_city = X_[:, j]
    
    states = np.hstack((states, last_city.reshape(-1, 1)))
    states_uq, cnt = np.unique(states, axis=0, return_counts=True)
    # cntn = cnt / cnt.sum()
    baseline = n_pareto_sol / states_uq.shape[0]
    n_critical = (cnt >= baseline).sum()
    
    
    layer = dd[i-1]    
    for suq in states_uq:
        state_found = False
        for sdd in layer:
            if np.array_equal(suq, sdd):
                state_found = True
                break
        assert state_found
        
    n_pareto += len(states_uq)
    n_nodes += len(layer)

    print(f"States: {len(layer)}, Pareto: {states_uq.shape[0]}, Critical: {n_critical}, Frac/Pareto: {states_uq.shape[0]/len(layer):.4f}, Frac/Critical: {n_critical/len(layer):.6f}")


States: 1, Pareto: 1, Critical: 0, Frac/Pareto: 1.0000, Frac/Critical: 0.000000
States: 14, Pareto: 11, Critical: 0, Frac/Pareto: 0.7857, Frac/Critical: 0.000000
States: 182, Pareto: 41, Critical: 0, Frac/Pareto: 0.2253, Frac/Critical: 0.000000
States: 1092, Pareto: 69, Critical: 0, Frac/Pareto: 0.0632, Frac/Critical: 0.000000
States: 4004, Pareto: 95, Critical: 0, Frac/Pareto: 0.0237, Frac/Critical: 0.000000
States: 10010, Pareto: 112, Critical: 0, Frac/Pareto: 0.0112, Frac/Critical: 0.000000
States: 18018, Pareto: 143, Critical: 0, Frac/Pareto: 0.0079, Frac/Critical: 0.000000
States: 24024, Pareto: 164, Critical: 0, Frac/Pareto: 0.0068, Frac/Critical: 0.000000
States: 24024, Pareto: 216, Critical: 0, Frac/Pareto: 0.0090, Frac/Critical: 0.000000
States: 18018, Pareto: 225, Critical: 0, Frac/Pareto: 0.0125, Frac/Critical: 0.000000
States: 10010, Pareto: 207, Critical: 0, Frac/Pareto: 0.0207, Frac/Critical: 0.000000
States: 4004, Pareto: 166, Critical: 0, Frac/Pareto: 0.0415, Frac/Criti

In [12]:
states_uq

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])